In [0]:
import gensim
import pandas as pd
import numpy as np
import csv
import random
import sklearn

import nltk 
from nltk.text import Text 
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer()

nltk.download("stopwords") 
from nltk.corpus import stopwords 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from google.colab import drive                  
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## word2vec + fasttext

загрузка модели

In [0]:
# если модель без тэгов
#model_file = '../data/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'

# model = Word2Vec.load(model_path)

In [0]:
# если модель с POS-тэггингом
model_file = '../data/tayga.vec'

model = KeyedVectors.load_word2vec_format(model_file, binary=False)

проверка наличия слова в словаре

In [0]:
lemma = 'черепаха_NOUN'


получение вектора слова

In [0]:
model['черепаха']

array([ 0.38691303, -0.01353064,  0.47982895,  0.24796614, -0.4995282 ,
       -0.07663023,  0.06268613,  0.3079075 ,  0.22344151,  0.20854878,
       -0.14401978, -0.34113285,  0.10848983, -0.41619024,  0.10033557,
        0.85966784, -0.13698106, -0.1241506 ,  0.06038719,  0.02368295,
       -0.16367738,  0.19001935, -0.1611998 , -0.25262153, -0.23922259,
        0.17140332,  0.05625665,  0.42271674,  0.26669613,  0.19677134,
        0.24412721, -0.21821369,  0.12846738, -0.06482669, -0.355308  ,
       -0.01725022,  0.00687153,  0.23292094,  0.43628576,  0.02504337,
        0.27711275,  0.04836041, -0.04329239,  0.16770369,  0.10105432,
       -0.11157424,  0.04214865,  0.14237887,  0.09776556, -0.00452495,
        0.14176252,  0.21383609,  0.2522228 , -0.44199   , -0.27218285,
        0.04738526, -0.31560647, -0.26865557, -0.01795514, -0.06383659,
       -0.04156402,  0.03212297,  0.23206615,  0.02170166,  0.02386148,
        0.43950433, -0.40797994,  0.13168378,  0.03637916,  0.04

In [0]:
# model[lemma]

получение вектора документа

In [0]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный_ADJ', 'замок_NOUN']

# создаем маски для векторов 
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model.wv:
        lemmas_vectors[idx] = model.wv[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

In [0]:
model_fasttext = KeyedVectors.load("/content/gdrive/My Drive/model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
with open('/content/gdrive/My Drive/quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as f:  
    spamreader = csv.reader(f)
    all_data = [row[1:] for row in spamreader][1:]
    corpus = [row[1] for row in all_data]

In [0]:
with open('/content/gdrive/My Drive/lemmatized.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    preprocessed_docs = content.split('\t')

In [0]:
docs_lists = [doc.split() for doc in preprocessed_docs]

In [0]:
def create_query(all_data):
    for_queries = [row[0] for row in all_data[:20]]
    raw_query = random.choice(for_queries)
    QUERY = preprocess_query(raw_query)
    return QUERY

In [0]:
def preprocess_query(raw_query):
    normalized = [pmm.normal_forms(x)[0] for x in raw_query.split() if x not in russian_stopwords]
    return normalized

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

In [0]:
%%time

fasttext_matrix = pd.DataFrame(columns = [i for i in range(300)])
for document in docs_lists[:20]:
  vecs_by_words = [model_fasttext[word] for word in document if word in model_fasttext.vocab]
  doc_vec = np.mean(vecs_by_words, axis = 0)
  fasttext_matrix = fasttext_matrix.append(pd.Series(doc_vec, index=fasttext_matrix.columns ), ignore_index=True)

CPU times: user 56.6 ms, sys: 0 ns, total: 56.6 ms
Wall time: 59 ms


### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   

In [0]:
QUERY = create_query(all_data)

In [0]:
QUERY

['означать',
 'президентство',
 'козырь',
 'нынешний',
 'иностранный',
 'студент-магистр',
 'визе',
 'f1']

In [0]:
query_words_vecs = [model_fasttext[word] for word in QUERY if word in model_fasttext.vocab]
QUERY_vec = np.mean(query_words_vecs, axis = 0)

In [0]:
QUERY_vec = QUERY_vec.reshape(1, -1)

In [0]:
cosine_fasttext_scores = []

for i in range(fasttext_matrix.shape[0]):
  docvec = np.array(fasttext_matrix.iloc[i]).reshape(1, -1)
  similarity = sklearn.metrics.pairwise.cosine_similarity(QUERY_vec, docvec)
  cosine_fasttext_scores.append(similarity[0][0])

In [0]:
cosine_fasttext_scores_sorted = sorted([(e,i) for i,e in enumerate(cosine_fasttext_scores)], reverse = True)
  
for item in list(cosine_fasttext_scores_sorted[:5]):
  print(all_data[item[1]][1], item[0], all_data[item[1]][2])   #посчитать точность по последней цифре тут

как президентство козыря повлияет на учеников в настоящее время в нас или планирует учиться у нас 0.73189545 1
каковы законы об изменении вашего статуса от студенческой визы до зеленой карты в нас, как они сравниваются с иммиграционными законами в Японии 0.66927856 0
который является лучшим институтом цифрового маркетинга в Пуне 0.6469707 0
что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад 0.6389768 0
что некоторые технические специалисты могут рассказать о долговечности и надежности ноутбуков и их компонентов 0.6140229 0
